In [1]:
import os

In [2]:
from pyspark.sql import Window
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [3]:
from unidecode import unidecode

In [4]:
from config import spark

22/05/02 13:17:55 WARN Utils: Your hostname, eniac resolves to a loopback address: 127.0.1.1; using 192.168.100.6 instead (on interface wlp3s0)
22/05/02 13:17:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/02 13:17:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/02 13:17:56 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [5]:
datalake = '/media/fabio/19940C2755DB566F/PAMepi/datalake/DATALAKE/'

In [6]:
version = '2022-04-19'

In [7]:
df = spark.read.csv(os.path.join(datalake, version, 'vacinacao'), header=True, sep=';')

In [8]:
columns = [
    'vacina_dataAplicacao',
    'paciente_id',
    'paciente_endereco_coIbgeMunicipio',
    'estabelecimento_municipio_codigo',
    'vacina_descricao_dose',
    'vacina_nome',
]

In [9]:
df = df.withColumnRenamed('vacina_dataAplicacao', 'date') \
    .withColumnRenamed('paciente_id', 'id') \
    .withColumnRenamed('paciente_endereco_coIbgeMunicipio', 'code_mun_ibge') \
    .withColumnRenamed('estabelecimento_municipio_codigo', 'estabelecimento') \
    .withColumnRenamed('vacina_descricao_dose', 'dose') \
    .withColumnRenamed('vacina_nome', 'dose_name')

In [10]:
df = df.withColumn('dose', F.udf(lambda dose: unidecode(dose).lower() if dose is not None else None)('dose'))
df = df.withColumn('dose_name', F.udf(lambda dose: unidecode(dose).lower() if dose is not None else None)('dose_name'))

In [11]:
df = df.withColumn('code_mun_ibge', F.when(F.col('code_mun_ibge').isNull(), F.col('estabelecimento')).otherwise(F.col('code_mun_ibge')))

In [12]:
df.select('date', 'id', 'code_mun_ibge', 'dose', 'dose_name').show(3, truncate=False)

+----------+----------------------------------------------------------------+-------------+-------+-----------------------------------------+
|date      |id                                                              |code_mun_ibge|dose   |dose_name                                |
+----------+----------------------------------------------------------------+-------------+-------+-----------------------------------------+
|2021-07-28|43f9ad3984ca1276c0557d43af4fd66a285dc907a76c9324b911d42029500250|410690       |2a dose|covid-19 astrazeneca/fiocruz - covishield|
|2021-04-29|dd84335d45c805792f1fcd8501145467859ae43d390cd74490a9af185a8ce21d|330455       |1a dose|covid-19 astrazeneca/fiocruz - covishield|
|2021-05-05|b44a2e37e070c83f005cdfdce73113d8382cd7ac1f1a9eb8e864730b8eba82c2|130260       |2a dose|covid-19 astrazeneca/fiocruz - covishield|
+----------+----------------------------------------------------------------+-------------+-------+-----------------------------------------+
only s

In [13]:
code_ibge = df.groupby('date', 'id').agg(F.first('code_mun_ibge').alias('code_mun_ibge'))
code_ibge.write.csv(os.path.join(datalake, version, 'preprocess', 'code_ibge_vac'), header=True, mode='overwrite')

In [ ]:
doses = df.groupby('date', 'id').pivot('dose').agg(F.first('dose_name').alias('dose_name'))
doses.write.csv(os.path.join(datalake, version, 'preprocess', 'doses_vac'), header=True, mode='overwrite')

In [7]:
code_ibge = spark.read.csv(os.path.join(datalake, version, 'preprocess', 'code_ibge_vac'), header=True)

In [9]:
code_ibge.show(3, truncate=False)

+----------+----------------------------------------------------------------+-------------+
|date      |id                                                              |code_mun_ibge|
+----------+----------------------------------------------------------------+-------------+
|1962-07-07|c1638657130a4eb994cc22296ae30576571c900436e04181ac768e9179ff3e74|530010       |
|1991-05-10|ff9273de3fd0f70556531ca7e1c222bcb98bea049bdf2f8d36be171921309b7d|421660       |
|2002-02-08|81b1f9c3049dbd8b8083d68fa1162c3509512f99739e4de1fd45f1d0714fece4|230125       |
+----------+----------------------------------------------------------------+-------------+
only showing top 3 rows



In [10]:
doses = spark.read.csv(os.path.join(datalake, version, 'preprocess', 'doses_vac'), header=True)

In [13]:
doses = doses.withColumn('dt_pattern', F.lit('2022-04-19'))

In [12]:
doses.show(1, truncate=False, vertical=True)

-RECORD 0------------------------------------------------------------------------------------------
 date                           | 2011-08-30                                                       
 id                             | 469287be5ff4c8a3c62d1f85f7faa4995dfed4c20ef318cadb1a0afc930887bf 
 1a dose                        | covid-19 pfizer - comirnaty                                      
 1a dose revacinacao            | null                                                             
 1o reforco                     | null                                                             
 2a dose                        | null                                                             
 2a dose revacinacao            | null                                                             
 2o reforco                     | null                                                             
 3a dose                        | null                                                             


In [14]:
doses = doses.withColumn(
    'num_dose_reg_VAC',
    F.when(
        (F.col('1a dose').isNotNull()) |
        (F.col('1o reforco').isNotNull()) |
        (F.col('2a dose').isNotNull()) |
        (F.col('2o reforco').isNotNull()) |
        (F.col('3a dose').isNotNull()) |
        (F.col('3o reforco').isNotNull()) |
        (F.col('4a dose').isNotNull()) |
        (F.col('5a dose').isNotNull()) |
        (F.col('revacinacao').isNotNull()) |
        (F.col('1a dose revacinacao').isNotNull()) |
        (F.col('2a dose revacinacao').isNotNull()) |
        (F.col('3a dose revacinacao').isNotNull()) |
        (F.col('dose inicial').isNotNull()) |
        (F.col('dose adicional').isNotNull()) |
        (F.col('reforco').isNotNull()) |
        (F.col('unica').isNotNull()) |
        (F.col('tratamento com uma dose').isNotNull()) |
        (F.col('tratamento com dezessete doses').isNotNull()),
        F.lit(1)
    )
).withColumn(
    'num_dose_pri_VAC',
    F.when(
        ((F.col('1a dose').isNotNull()) |
        (F.col('dose inicial').isNotNull())) &
        (F.col('2a dose').isNull()),
        F.lit(1)
    )
).withColumn(
    'num_dose_sec_uni_VAC',
    F.when(
        (F.col('2a dose').isNotNull()) |
        (F.col('unica').isNotNull()),
        F.lit(1)
    )
).withColumn(
    'num_dose_reinforcment_VAC',
    F.when(
        (F.col('3a dose').isNotNull()) |
        (F.col('4a dose').isNotNull()) |
        (F.col('5a dose').isNotNull()) |
        (F.col('dose adicional').isNotNull()) |
        (F.col('1a dose revacinacao').isNotNull()) |
        (F.col('2a dose revacinacao').isNotNull()) |
        (F.col('3a dose revacinacao').isNotNull()) |
        (F.col('dose adicional').isNotNull()) |
        (F.col('reforco').isNotNull()) |
        (F.col('1o reforco').isNotNull()) |
        (F.col('2o reforco').isNotNull()) |
        (F.col('3o reforco').isNotNull()),
        F.lit(1)
    )
).withColumn(
    'num_dose_non_reg_VAC',
    F.when(F.col('dose').isNotNull(), F.lit(1))
).withColumn(
    'total_dose_VAC',
    F.col('num_dose_reg_VAC')+F.col('num_dose_non_reg_VAC')
)#.show(1, vertical=True, truncate=False)

In [15]:
doses = doses.withColumn(
    'num_vac_type_1_VAC',
    F.when(
        (F.col('1a dose').contains('astrazeneca')) |
        (F.col('1o reforco').contains('astrazeneca')) |
        (F.col('2a dose').contains('astrazeneca')) |
        (F.col('2o reforco').contains('astrazeneca')) |
        (F.col('3a dose').contains('astrazeneca')) |
        (F.col('3o reforco').contains('astrazeneca')) |
        (F.col('4a dose').contains('astrazeneca')) |
        (F.col('5a dose').contains('astrazeneca')) |
        (F.col('revacinacao').contains('astrazeneca')) |
        (F.col('1a dose revacinacao').contains('astrazeneca')) |
        (F.col('2a dose revacinacao').contains('astrazeneca')) |
        (F.col('3a dose revacinacao').contains('astrazeneca')) |
        (F.col('dose inicial').contains('astrazeneca')) |
        (F.col('dose adicional').contains('astrazeneca')) |
        (F.col('reforco').contains('astrazeneca')) |
        (F.col('unica').contains('astrazeneca')) |
        (F.col('tratamento com uma dose').contains('astrazeneca')) |
        (F.col('tratamento com dezessete doses').contains('astrazeneca')),
        F.lit(1)
    )
).withColumn(
    'num_vac_type_2_VAC',
    F.when(
        (F.col('1a dose').contains('coronavac')) |
        (F.col('1o reforco').contains('coronavac')) |
        (F.col('2a dose').contains('coronavac')) |
        (F.col('2o reforco').contains('coronavac')) |
        (F.col('3a dose').contains('coronavac')) |
        (F.col('3o reforco').contains('coronavac')) |
        (F.col('4a dose').contains('coronavac')) |
        (F.col('5a dose').contains('coronavac')) |
        (F.col('revacinacao').contains('coronavac')) |
        (F.col('1a dose revacinacao').contains('coronavac')) |
        (F.col('2a dose revacinacao').contains('coronavac')) |
        (F.col('3a dose revacinacao').contains('coronavac')) |
        (F.col('dose inicial').contains('coronavac')) |
        (F.col('dose adicional').contains('coronavac')) |
        (F.col('reforco').contains('coronavac')) |
        (F.col('unica').contains('coronavac')) |
        (F.col('tratamento com uma dose').contains('coronavac')) |
        (F.col('tratamento com dezessete doses').contains('coronavac')),
        F.lit(1)
    )
).withColumn(
    'num_vac_type_3_VAC',
    F.when(
        (F.col('1a dose').contains('pfizer')) |
        (F.col('1o reforco').contains('pfizer')) |
        (F.col('2a dose').contains('pfizer')) |
        (F.col('2o reforco').contains('pfizer')) |
        (F.col('3a dose').contains('pfizer')) |
        (F.col('3o reforco').contains('pfizer')) |
        (F.col('4a dose').contains('pfizer')) |
        (F.col('5a dose').contains('pfizer')) |
        (F.col('revacinacao').contains('pfizer')) |
        (F.col('1a dose revacinacao').contains('pfizer')) |
        (F.col('2a dose revacinacao').contains('pfizer')) |
        (F.col('3a dose revacinacao').contains('pfizer')) |
        (F.col('dose inicial').contains('pfizer')) |
        (F.col('dose adicional').contains('pfizer')) |
        (F.col('reforco').contains('pfizer')) |
        (F.col('unica').contains('pfizer')) |
        (F.col('tratamento com uma dose').contains('pfizer')) |
        (F.col('tratamento com dezessete doses').contains('pfizer')),
        F.lit(1)
    )
).withColumn(
    'num_vac_type_4_VAC',
    F.when(
        (F.col('1a dose').contains('janssen')) |
        (F.col('1o reforco').contains('janssen')) |
        (F.col('2a dose').contains('janssen')) |
        (F.col('2o reforco').contains('janssen')) |
        (F.col('3a dose').contains('janssen')) |
        (F.col('3o reforco').contains('janssen')) |
        (F.col('4a dose').contains('janssen')) |
        (F.col('5a dose').contains('janssen')) |
        (F.col('revacinacao').contains('janssen')) |
        (F.col('1a dose revacinacao').contains('janssen')) |
        (F.col('2a dose revacinacao').contains('janssen')) |
        (F.col('3a dose revacinacao').contains('janssen')) |
        (F.col('dose inicial').contains('janssen')) |
        (F.col('dose adicional').contains('janssen')) |
        (F.col('reforco').contains('janssen')) |
        (F.col('unica').contains('janssen')) |
        (F.col('tratamento com uma dose').contains('janssen')) |
        (F.col('tratamento com dezessete doses').contains('janssen')),
        F.lit(1)
    )
)

In [16]:
doses = doses.withColumn(
    'num_ind_incom_delay_less2_VAC',
    F.when(
        F.col('num_dose_pri_VAC').isNotNull(),
        F.when(
            F.months_between(F.col('dt_pattern'), F.col('date')) < 2,
            F.lit(1)
        )
    )
).withColumn(
    'num_ind_incom_delay_2_6_VAC',
    F.when(
        F.col('num_dose_pri_VAC').isNotNull(),
        F.when(
            (F.months_between(F.col('dt_pattern'), F.col('date')) >= 2) &
            (F.months_between(F.col('dt_pattern'), F.col('date')) < 6),
            F.lit(1)
        )
    )
).withColumn(
    'num_ind_incom_delay_more6_VAC',
    F.when(
        F.col('num_dose_pri_VAC').isNotNull(),
        F.when(
            F.months_between(F.col('dt_pattern'), F.col('date')) > 6,
            F.lit(1)
        )
    )
)

In [17]:
doses = doses.withColumn(
    'num_dose_sec_uni_delay_less2_VAC',
    F.when(
        F.col('num_dose_sec_uni_VAC').isNotNull(),
        F.when(
            F.months_between(F.col('dt_pattern'), F.col('date')) < 2,
            F.lit(1)
        )
    )
).withColumn(
    'num_dose_sec_uni_delay_2_6_VAC',
    F.when(
        F.col('num_dose_sec_uni_VAC').isNotNull(),
        F.when(
            (F.months_between(F.col('dt_pattern'), F.col('date')) >= 2) &
            (F.months_between(F.col('dt_pattern'), F.col('date')) < 6),
            F.lit(1)
        )
    )
).withColumn(
    'num_dose_sec_uni_delay_more6_VAC',
    F.when(
        F.col('num_dose_sec_uni_VAC').isNotNull(),
        F.when(
            F.months_between(F.col('dt_pattern'), F.col('date')) > 6,
            F.lit(1)
        )
    )
)

In [18]:
doses = doses.withColumn(
    'num_ind_com_atu_delay_less2_VAC',
    F.when(
        F.col('num_dose_reinforcment_VAC').isNotNull(),
        F.when(
            F.months_between(F.col('dt_pattern'), F.col('date')) < 2,
            F.lit(1)
        )
    )
).withColumn(
    'num_ind_com_atu_delay_2_6_VAC',
    F.when(
        F.col('num_dose_reinforcment_VAC').isNotNull(),
        F.when(
            (F.months_between(F.col('dt_pattern'), F.col('date')) >= 2) &
            (F.months_between(F.col('dt_pattern'), F.col('date')) < 6),
            F.lit(1)
        )
    )
).withColumn(
    'num_ind_com_atu_delay_more6_VAC',
    F.when(
        F.col('num_dose_reinforcment_VAC').isNotNull(),
        F.when(
            F.months_between(F.col('dt_pattern'), F.col('date')) > 6,
            F.lit(1)
        )
    )
)

In [24]:
columns = doses.columns[22:]

In [26]:
doses.join(code_ibge, how='left', on=['date', 'id']).groupby('date', 'code_mun_ibge') \
.agg(
    *[F.count(col).astype('int').alias(col) for col in columns]
).write.csv(os.path.join(datalake, version, 'preprocess', 'vacinacao'), header=True, mode='overwrite')

22/05/02 22:23:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/05/02 22:23:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/05/02 22:23:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/05/02 22:23:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/05/02 22:23:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/05/02 22:23:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/05/02 22:23:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/05/02 22:23:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/05/02 22:25:08 WARN RowBasedKeyValueBatch: Calling spill() on